# Partie 1

## Modules

In [1]:
import openpyxl
from read_input import read_input

from gurobipy import *
import math
import numpy 
import time

## Lecture des données

In [2]:
start_time = time.time()

ville="Italy" #  changer 
ressources,ressources_unavailabilities,tasks,tasks_unavailabilities = read_input(1,ville)
# "Bordeaux", "Finland", "GuineaGolf", "Italy", "Poland"

## Structures de données

In [3]:
def dist(lat1, long1, lat2, long2):
    """
    fonction renvoyant la distance en kms entre deux points décrits par leur latitude et longitude
    """
    if lat1==lat2 and long1==long2:
        return 0
    return(numpy.arccos(numpy.sin(2*numpy.pi/360*lat1)*numpy.sin(2*numpy.pi/360*lat2)+numpy.cos(2*numpy.pi/360*lat1)*numpy.cos(2*numpy.pi/360*lat2)*numpy.cos(2*numpy.pi/360*(long2-long1)))*6371)

#ACOS(SIN(lat1)*SIN(lat2)+COS(lat1)*COS(lat2)*COS(lon2-lon1))*6371
v=5/6

L_ressources = [j for j in ressources] # tableau des objets ressources 
L_tasks = [i for i in tasks] # tableau des objets taches
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
print(dict_pos)

#distance[a][b] donne la distance de la tâche a vers la tâche b
distances = [[dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude) for b in range(number_of_tasks)] for a in range(number_of_tasks)]

#distance_m[j][i] donne la distance de la maison de j la tâche i
distances_m = [[dist(ressources[L_ressources[j]].latitude,ressources[L_ressources[j]].longitude, tasks[L_tasks[i]].latitude, tasks[L_tasks[i]].longitude) for i in range(number_of_tasks)] for j in range(number_of_ressources)]
print(distances_m)

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

{0: (45.275598, 8.742639), 1: (45.29162, 9.293758), 2: (45.354836, 9.200242), 3: (45.186421, 9.48472), 4: (45.213584, 9.08807), 5: (45.255491, 9.596659), 6: (45.294706, 9.834174), 7: (45.355288, 9.001617), 8: (45.356011, 9.555898), 9: (45.482644, 9.405359), 10: (45.495033, 9.778958), 11: (45.71057, 9.359627), 12: (45.54103, 9.642563), 13: (45.649801, 9.796366), 14: (45.101531, 9.345792), 15: (44.936769, 9.383681), 16: (44.980562, 9.420916), 17: (44.965977, 9.420363), 18: (44.931524, 9.265948), 19: (44.993501, 8.867372), 20: (45.547869, 9.058474)}
[[51.90907219190567, 11.174905898034892, 21.308610143516553, 7.838929448103234, 24.628297019686936, 15.586954637607084, 34.67472314388933, 34.458633970856134, 18.804620229628185, 28.528415835766825, 41.97219147298, 53.972706930066956, 39.74052189836434, 56.27141255007598, 14.534954103463928, 32.20347989415279, 27.342058702565335, 28.959397648976484, 34.45271428676269, 49.28677591982298, 44.72055437699315], [51.90907219190567, 11.17490589803489

## Variables de décision

In [4]:
### Create Model
m = Model("trs0")

M = 1440

print(number_of_ressources)
print(number_of_tasks)

### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

# Is the worker used ?
u = {r: m.addVar(vtype = GRB.BINARY, name = f'u_{r}') for r in range(number_of_ressources)}

# starting task of worker
d = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'd_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}
# ending task of worker
f = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'f_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

#variables de linéarisation
#H = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'h_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
#P = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'p_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}


Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-27
3
21


## Contraintes

In [5]:
# une personne est obligée de se déplacer (pas d'arrêtes de bouclage sur un point du graphe)
C4_2 = [m.addConstr(y[(r,a,a)] == 0) for r in range(number_of_ressources) for a in range(number_of_tasks)]

In [6]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

# One ressource per task
C1 = [m.addConstr(nbRpT[t] == 1) for t in range(number_of_tasks)]

# Works only if all tasks are feasable
#nbRpTq = {t: LinExpr() for t in range(number_of_tasks)}
#for t in range(number_of_tasks):
#  for r in  range(number_of_ressources):
#    if ressources[L_ressources[r]].skill==tasks[L_tasks[t]].skill and ressources[L_ressources[r]].level>=tasks[L_tasks[t]].level:
#      nbRpTq[t]+=x[(t,r)]

# Qualified technicians
#C2 = [m.addConstr(nbRpTq[t] >= 1) for t in range(number_of_tasks)]

#C1 = [m.addConstr(quicksum(x[(t,r)] for r in range(number_of_ressources)) == 1 for t in range(number_of_tasks))]



In [7]:
nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
     nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

 # Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

In [8]:
nbRpTl =[ LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[t]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

In [9]:
# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks)) + d[(j,i)] == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks)) + f[(j,i)] == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

In [10]:
# Non overlapping tasks previous version

# C6 = [m.addConstr(H[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C7 = [m.addConstr(H[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C8 = [m.addConstr(H[(r,a,b)] - s[a]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C9 = [m.addConstr(H[(r,a,b)] - s[a] + 1440*(1-H[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]

# C10 = [m.addConstr(P[(r,a,b)] >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C11 = [m.addConstr(P[(r,a,b)] - 1440 * y[(r,a,b)]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C12 = [m.addConstr(P[(r,a,b)] - s[b]  <=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C13 = [m.addConstr(P[(r,a,b)] - s[b] + 1440*(1-P[(r,a,b)])  >=0) for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)]# if a != b]
# C14Sum =[[LinExpr() for a in range(number_of_tasks)] for b in range(number_of_tasks)]
# for a in range(number_of_tasks):
#   for b in range(number_of_tasks):
#     for j in  range(number_of_ressources):
#       C14Sum[a][b]+=H[(j,a,b)] - P[(j,a,b)] + y[(j,a,b)] * (tasks[L_tasks[a]].duration + distances[a][b]/v)

# C14 = [m.addConstr(C14Sum[a][b] <= 0) for a in range(number_of_tasks) for b in range(number_of_tasks)]

In [11]:
# non overlapping tasks
print(distances[0][1]/v)
print(tasks[L_tasks[0]].duration)
C6 = [m.addConstr(s[b] >= s[a] + tasks[L_tasks[a]].duration + distances[a][b]/v -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for a in range(number_of_tasks) for b in range(number_of_tasks)]

51.78521279106653
30


In [12]:
# la premiere tache commence apres le début de la journée
#j,i=0,3
#print(ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)].x) + distances_m[j][i]/v)
#M=10E2
C11 = [m.addConstr(s[i] >= ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)]) + distances_m[j][i]/v) for i in range(number_of_tasks) for j in range(number_of_ressources)]

In [13]:
# la derniere tache finit avant la fin de la journee
C12 = [m.addConstr(ressources[L_ressources[j]].workingEnd >= s[i]+tasks[L_tasks[i]].duration + distances_m[j][i]/v -M*(1-x[(i,j)])) for i in range(number_of_tasks) for j in range(number_of_ressources)]

In [14]:
# une personne doit partir de chez elle si elle travaille ce jour la
C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]
# une personne doit revenir chez elle si elle travaille ce jour la
C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks)) == u[j]) for j in range(number_of_ressources)]

# contraintes pour lier x et u
C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) <= u[j]*M) for j in range(number_of_ressources)]
#C10 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) >= u[j]) for j in range(number_of_ressources)]

# contraintes pour lier x et (d et f)
#C13 = [d[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]
#C14 = [f[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]



In [15]:
# C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks)) == 1) for j in range(number_of_ressources)]
# C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks)) == 1) for j in range(number_of_ressources)]

# # C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) <= u[j]*M) for j in range(number_of_ressources)]
# # C10 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks)) >= u[j]) for j in range(number_of_ressources)]

# C13 = [d[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]
# C14 = [f[(j,i)]<=x[(i,j)] for i in range(number_of_tasks) for j in range(number_of_ressources)]


In [16]:
# contrainte évitant d'avoir des cycles dans les graphes (permet que les tâches soient reliées entre elles)
C15 = [m.addConstr(y[(j,a,b)] + y[(j,b,a)] <=1) for a in range(number_of_tasks) for b in range(number_of_tasks) for j in range(number_of_ressources)]
C16 = [m.addConstr(d[(j,a)] + f[(j,a)] <=1) for a in range(number_of_tasks) for j in range(number_of_ressources)]

In [17]:
# contraintes permettant de respecter les horaires d'ouverture et de fermeture des taches
C17 = [m.addConstrs(s[i] >= tasks[L_tasks[i]].openingTime  -M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources))) for i in range(number_of_tasks))]
C18 = [m.addConstrs(tasks[L_tasks[i]].closingTime >= s[i] + tasks[L_tasks[i]].duration - M*(1-quicksum(x[(i,j)] for j in range (number_of_ressources)))for i in range(number_of_tasks))]

## Fonction objectif et résolution

In [18]:
# -- Ajout de la fonction objectif --
couts=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      couts+=y[(j,a,b)]*distances[a][b]/v
    couts+=d[(j,a)]*distances_m[j][a]/v
    couts+=f[(j,a)]*distances_m[j][a]/v
m.setObjective(couts, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

# -- Affichage en mode texte du PL --
m.display()

# -- Résolution --
m.optimize()

# -- Vérification du statut et Affichage (le cas échéant) des solutions --
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")


# for j in range(number_of_tasks):
#   for i in range(number_of_ressources):
#     print(x[(j,i)].x,end=' ')
#   print("\n")

# print("\n\n\n")


# -- Affichage de la solution --
#print("La solution optimale est (d,q) = {} \navec pour valeur de l'objectif z = {}€".format((D.x, Q.x), m.objVal))
t=open("Solution"+ville+"V1ByV4.txt",'w'.format(ville))
res="taskId;performed;employeeName;startTime;\n"
for j in range(number_of_tasks):
  p=False
  for i in range(number_of_ressources):
    if round(x[(j,i)].x):
      res+="{};{};{};{};\n".format(L_tasks[j],1,L_ressources[i],int(s[j].x))
      p=True
  if not p:
    res+="{};;;;\n".format(L_tasks[j])
print(res)
t.write(res)
t.close()

print("###########################")

for a in range(number_of_tasks):
  for b in range(number_of_tasks):
    j=0
    if  round(y[(j,a,b)].x)==1:
      print("j{} - a{} - b{} - y{} // ".format(j, a,b,y[(j,a,b)].x), end="")

#7 - 4 - 2 - 1 - 7
#8 - 6 - 8

print('\n') 
print("###########################")

for a in range(number_of_tasks):
  j=0
  if  round(d[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,d[(j,a)].x), end="")


print('\n\n')
for a in range(number_of_tasks):
  j=0
  if  round(f[(j,a)].x)==1:
    print("j{} - a{} - d{} // ".format(j, a,f[(j,a)].x), end="")

# 3 - 5 - 2 - 9 - 7 - 4 - 1 - 0
# 6-8 okok

interval = time.time() - start_time

taskId;performed;employeeName;startTime;
T1;1;Carolina;658;
T2;1;Mattia;889;
T3;1;Mattia;846;
T4;1;Carolina;984;
T5;1;Mattia;794;
T6;1;Paolo;499;
T7;1;Paolo;562;
T8;1;Carolina;601;
T9;1;Paolo;620;
T10;1;Paolo;683;
T11;1;Paolo;874;
T12;1;Paolo;754;
T13;1;Paolo;819;
T14;1;Paolo;972;
T15;1;Mattia;612;
T16;1;Mattia;675;
T17;1;Carolina;910;
T18;1;Carolina;863;
T19;1;Carolina;807;
T20;1;Carolina;728;
T21;1;Carolina;534;

###########################
j0 - a2 - b1 - y1.0 // j0 - a4 - b2 - y1.0 // j0 - a14 - b15 - y1.0 // j0 - a15 - b4 - y1.0 // 

###########################
j0 - a14 - d1.0 // 


j0 - a1 - d1.0 // 

In [19]:
print("temps de calcul : ", interval)
print("valeur de la fonction objectif : ", m.ObjVal)

temps de calcul :  2000.8830354213715
valeur de la fonction objectif :  631.1513807817946


## Vérification de la solution

In [20]:
# affichage de la carte
import folium

lat_tot,long_tot=0,0
for a in range(number_of_tasks):
  lat_tot+=tasks[L_tasks[a]].latitude
  long_tot+=tasks[L_tasks[a]].longitude
mid_latitude = lat_tot/number_of_tasks
mid_longitude = long_tot/number_of_tasks
carte=folium.Map(location=[mid_latitude, mid_longitude],zoom_start=10)
for a in range(number_of_tasks):
  folium.Marker([tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude],icon=folium.Icon(color='black'),popup=str(a)).add_to(carte)
list_colors = ['darkgreen', 'blue', 'red', 'green', 'darkpurple', 'lightred', 'purple', 'pink', 'orange', 'darkblue', 'gray', 'darkred']
number_of_colors = len(list_colors)
print(number_of_colors)
counter = 0
for r in range(number_of_ressources):
  color_r = list_colors[counter % number_of_colors]
  folium.Marker([ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude],icon=folium.Icon(color=color_r),popup="pers").add_to(carte)
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      if round(y[(r,a,b)].x) == 1:
        line_ab = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude),(tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude)]
        folium.PolyLine(line_ab, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(d[(r,a)].x) == 1:
      line_home_t = [(ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude), (tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude)]
      folium.PolyLine(line_home_t, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(f[(r,a)].x) == 1:
      line_t_home = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude), (ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude)]
      folium.PolyLine(line_t_home, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
  counter+=1

carte.save('Cartographie/Cartev1.html')

carte

12


In [24]:
import plotly.express as px
import pandas as pd

# plotting the working tasks
liste=[]
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            start=int(s[j].x)
            end=(int(s[j].x)+tasks[L_tasks[j]].duration)
            liste.append(dict(Task=L_ressources[i], Start=start, Finish=end, Resource="executing task",J="tâche "+str(j+1)))

# plotting the transports
for a in range(number_of_tasks):
    for b in range(number_of_tasks):
        for j in range(number_of_ressources):
            if round(y[(j,a,b)].x):
                start=(int(s[a].x)+tasks[L_tasks[a]].duration)
                end=(start+distances[a][b]/v)
                liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="moving",J="trajet "+str(a+1)+"-"+str(b+1)))

# plotting the first transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(d[(j,a)].x):
            start=(s[a].x-distances_m[j][a]/v)
            end=s[a].x
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J="trajet maison -"+str(a+1)))

# plotting the last transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(f[(j,a)].x):
            start=(s[a].x + tasks[L_tasks[i]].duration)
            end=(start+ distances_m[j][a]/v)
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J=str(a+1)+"- trajet maison"))

df = pd.DataFrame(liste)
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task",text="J")

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
fig.show()

In [22]:
# check que les techniciens ont le bon niveau et les bonnes compétences pour les tâches exécutées
a = True
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            # check que le skill est bon
            if ressources[L_ressources[i]].skill!=tasks[L_tasks[i]].skill:
                print("fail r,t ",i,j)
                a = False
            # check que le level est bon
            if ressources[L_ressources[i]].level < tasks[L_tasks[j]].level:
                print("fail r,t ",i,j)
                a = False
if a :
    print("Les techniciens ont bien les bonnes compétences et le bon niveau pour toutes les tâches exécutées")
            
            

Les techniciens ont bien les bonnes compétences et le bon niveau pour toutes les tâches exécutées


In [23]:
# check que les contraintes d'ouverture et de fermeture des tâches sont respectées (à réécrire pour la v2)
a = True
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            if s[i].x < tasks[L_tasks[i]].openingTime:
                print("fail r,t ",i,j)
                a = False
            if tasks[L_tasks[i]].closingTime < s[i].x + tasks[L_tasks[i]].duration :
                print("fail r,t ",i,j)
                a = False

            
            
if a :
    print("Les contraintes d'ouverture et de fermeture des tâches sont respectées")


Les contraintes d'ouverture et de fermeture des tâches sont respectées



coder convexite

tout bencher

rajouter des checks sur les nouvelles contraintes, modifier l'emploi du temps (v2)

check contraites ouverture et fermeture des taches (recoder en v2)

coder qqes contraintes v2